In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import plotly.express as px

# Description:  
This Notebook is the example of Small analytical work on Test Data and Syntetic examples

# Structure:
1. [Task_1 Dashboard Creation](#task_1)
    - [Data reading and processing](#data-reading-and-processing)
    - [Visualization funnel](#visualization-funnel)
    - [Discussion](#discussion)
        - [How Dashboard can be improved](#how-dashboard-can-be-improved)
2. [Task_2 A/B test design](#task2-ab-test-design)
    - [Assumptions](#assumptions)
    - [Hypothesis](#hypothesis)
    - [Test](#test)
        - [Design](#design)
        - [Results processing](#обработка-результатов)

# Task_1
Company - Online Retailer  
Now company starts to work on increasing Conversion on Web-site  
Therefore, we want to monitor Conversion on main Stages of Customer Journey. We will create Conversion Funnel Dashboard based on Users Clicks Data.  
In the current project we consider specific Landing Wb page on Our main Web-site
Therefore, main stages that will be considered:
1. User on main web-page action
1. Transfer on considered Landing page
2. Click on Copy link button
3. Registration
4. Click on Pay button
5. Successful payment 

## Data Reading and Processing

Actions from Web page were collected to the .csv file:

In [3]:
# Reading Web page data:
web_page_data = pd.read_csv('./data/web_page_data.csv')
web_page_data.head(2)

,localUserId,dateTime,utm_campaign,utm_source,utm_medium,os,browser,blockid,event
0,1a8f3eb1-414b-45e8-8ccb-79043494ef68,2022-02-27 23:28:23.673 UTC,NaN,visitka,NaN,Android,Instagram,NaN,register-confirm-code-success
1,1a8f3eb1-414b-45e8-8ccb-79043494ef68,2022-02-27 23:29:10.436 UTC,NaN,visitka,NaN,Android,Instagram,NaN,editor.add.click


Web page data stores following information about each event:
- localUserId - id of user in Web Site DataBase
- dateTime - datetime string of action
- utm_campaign - name of marketing campaign
- utm_source - source of traffic
- utm_medium - name of campaign type
- os - Operational System of user
- browser - user's browser
- blockid - identificator of click, which was changed
- event - event type

In dashboard we decided to use following filtes:
- Stage (event type)
- Date of event
- Time of event
- Browser
- OS  

Local user Id will be used to Count number of Actions  

therefore, other columns may de droped for current task

In [7]:
# drop columns, which will not be used in analysis:
web_page_data.drop(columns=['utm_campaign',
                            'utm_source',
                            'utm_medium',
                            'blockid'], inplace=True)

Event column stores technical names which are not intuitive for analysis, therefore we need to load dictonary table with names of events:

In [10]:
# reading events descritption table:
events_descr = pd.read_excel('./data/events_types.xlsx', sheet_name='desc')
events_descr.head(3)

,event,event_descr
0,landing.unique-visit,Transfer on considered Landing page
1,register-confirm-code-success,Registration
2,editor.add.click,Click on add button


Then join, description to Web_page_data  
at this stage join is left, to ba able to analyze events on other stages
however, later other stages will be filtered

In [12]:
# join events descr:
data = pd.merge(web_page_data, events_descr, how='left', on='event')

For Analysis, we will separate Datetime column on Date and time columns  
Time column will be rounded to 30 minutes periods

In [13]:
# convert datetime column to datetime format and separate date and time (round to 30 min)
data['event_date_time'] = pd.to_datetime(data['dateTime'])
data['data'] = data['event_date_time'].dt.date
data['time'] = data['event_date_time'].dt.round('30min').dt.time

In [14]:
#save for analysis:
data.to_csv('./data/data_final.csv')

In [18]:
# columns in dataset
list(data.columns)

['localUserId',
 'dateTime',
 'os',
 'browser',
 'event',
 'event_descr',
 'event_date_time',
 'data',
 'time']

### Visualization funnel:

In [19]:
stage_list = ['User on main web-page action',
              'Transfer on considered Landing page',
              'Click on Copy link button',
              'Registration',
              'Click on Pay button',
              'Successful payment']

# filter only specific stages:
data = data[data['event_descr'].isin(stage_list)]

Calculate number of events on each Stage:

In [20]:
count_events = data.groupby('event_descr', as_index=False).agg({
    'localUserId': 'count'
})
count_events.columns = ['stage', 'count_events']
count_events.sort_values(by='count_events', ascending=False, inplace=True)
count_events

,stage,count_events
5,User on main web-page action,16562
4,Transfer on considered Landing page,8419
0,Click on Copy link button,2706
2,Registration,2404
1,Click on Pay button,94
3,Successful payment,24


Build funnel visualization:

In [21]:
fig = px.funnel(count_events, x='count_events', y='stage')
fig.show()

### Discussion

Such Dashboard may be usefull to evaluate Conversion between Stages to the Target event (Successful payment)  
- To increase efficiency of this Dashboard we need to clearly understand Customer Journey

In that example let us consider the folliwing Schema:

![](./images/client_journey.png)

Considering such Customer Journey, Created Funnel Chart may let us find insights about what stages can be "bottle neck" in our Sales process

### How Dashboard can be improved

- Data quality:  
    We can not make decisions based on data which we do not trust. That is why it is crusial to ensure that data is free of Duplicates or Anomalys and other data quality issues, which can distort results
- Add new data features:  
    Entrance of new data features (dimensions) (like geograpical data etc.), and also Measures (Ticket Amount and so on) may help to find new insights in data

# Task_2 A/B Test Design

Company - creates "Mini Sites" for users.
Each mini site created on the platform has Indication that references Company's site, which is now disegned in Blue color  
![Option A](./images/Option_A.png)
This feature allows product to gain new users   
However it may also infuence result for current users:
- For example. Massage Salon has created their Mini Site and then manager saw bright Blue design of bottom of web site and reject publishing

Consider the possible Customer Journey:  

![](./images/client_journey_mini_site.png)

## Assumptions

### Conversion

In [16]:
df_base = pd.DataFrame({
    'Stage in Journey': ['QR scan', 'Site creation', 'Final observation', 'Publish'],
    'Conversion Rate, %': [30, 60, 20, 10]
})
df_base

,Stage in Journey,"Conversion Rate, %"
0,QR scan,30
1,Site creation,60
2,Final observation,20
3,Publish,10


In [18]:
overall_conversion = \
df_base['Conversion Rate, %'][0]/100 * \
    df_base['Conversion Rate, %'][1]/100 *\
        df_base['Conversion Rate, %'][2]/100 * \
            df_base['Conversion Rate, %'][3]/100 * 100

print(f'Overall conversion rate = {overall_conversion} %')

Overall conversion rate = 0.36 %


Assume:
- Number of unique users : 10 000 each month

- Conversion Rate A (base case) = 10 %
- Conversion Rate B (studied option) = 13 %
- Significance level = 5 % (alpha)
- Statistical power = 80 % (betta)

## Hypothesis
    -  Null Hypothesis - each options have same efficiency
    -  Alternative Hypothesis - Options A and B have significantly different efficiency
- By creation of less distructing disign of site bottom part users like design of their mini sites more, which brings us to higher conversion on last stage
-  MDE - minimum detectable effect assume = **3 %**

## Metrics:
It is very important to choose proper metric, since it will influence Test timing and interpretation
- In current example it will be beneficial to choose Conversion Rate on the last stage of Customer Journey (between viewing final version of mini site and clicking "Publish")
    - The closer metric to the Interested stage the lower time will be required to produce statistically significant result

## Test
- We will conduct **two-tiled test** since we consider deviation from base case to the both sides (lower and higher conversion)

### Design:

In [3]:
Z_alpha = 0.05
Z_betta = 0.8
CR_a = 0.2
CR_b = 0.26
n_required_1_tiled = (st.norm.ppf(Z_alpha) + st.norm.ppf(1-Z_betta))**2 *\
                 (CR_a * (1-CR_a) + CR_b * (1-CR_b)) \
                  / (CR_a - CR_b) ** 2
n_required_2_tiled = (st.norm.ppf(Z_alpha/2) + st.norm.ppf(1-Z_betta))**2 *\
                 (CR_a * (1-CR_a) + CR_b * (1-CR_b)) \
                  / (CR_a - CR_b) ** 2

n_required = round(n_required_2_tiled) * 2
print(f'Size of data required for test: {n_required}')

Size of data required for test: 1536


In [6]:
number_of_users_each_month = 10000
number_of_users_per_day = number_of_users_each_month / 30
required_time_days = round(n_required / number_of_users_per_day)
print(f'Time required for test = {required_time_days} days')

Time required for test = 5 days


## Results interpretation

Let us assume that we have already conducted designed test and received following results:

In [18]:
# Population
n_a = n_required // 2 + 1
n_b = n_required // 2 + 1

# Conversion rate
CR_a = 0.1
CR_b = 0.13

# Variance
var_a = CR_a * (1-CR_a) 
var_b = CR_b * (1-CR_b)

conversions_a = CR_a * n_a
conversions_b = CR_b * n_b

print('N - a: {:0.0f} , b: {:0.0f}'.format(n_a, n_b))
print('CR (convcersion rate) - a: {:0.4f} , b: {:0.4f}'.format(CR_a, CR_b))
print('Conversions -  a: {:0.0f} , b: {:0.0f}'.format(conversions_a, conversions_b))
print('Variance -  a: {:0.4f} , b: {:0.4f}'.format(var_a, var_b))

# Create combined random variable S
mean_a = CR_a
mean_b = CR_b
S_mean = mean_b - mean_a
S_var = (var_a/n_a) + (var_b/n_b)

print('------------')
Z_score = S_mean / np.sqrt(S_var)
print('Z-score: {:0.4f}'.format(Z_score))

print()
import scipy.stats as st

p_value_1_tail = 1-st.norm.cdf(Z_score)
p_value_2_tail = p_value_1_tail * 2

print('p-value: {:0.6f}'.format(p_value_2_tail))

N - a: 769 , b: 769
CR (convcersion rate) - a: 0.1000 , b: 0.1300
Conversions -  a: 77 , b: 100
Variance -  a: 0.0900 , b: 0.1131
------------
Z-score: 1.8460

p-value: 0.064894


In [19]:
print(f'Certainty in results significance: {round(1 - p_value_2_tail, 4) * 100} %')

Certainty in results significance: 93.51 %
